In [ ]:
import glob
from tqdm import tqdm
import os
import pandas as pd
import cv2

## paths:
5. Text files containing name and cordinates of images.
1. csv file consisting of image name and its corresponding "xmin","ymin","xmax", "ymax" cordinates
2. dir containing input images
3. white images created by input images consisting of 1 pixels
4. masked images consisting of black pixels wherever text is found(that will be created after preprocessing)

In [ ]:
csv_path = os.path.join(Project_path, "data/csv_files/data_xml.csv")
image_dir = os.path.join(Project_path, "data/input_images/")
images_white_dir = os.path.join(Project_path, "data/white_images/")
labelled_images_masked = os.path.join(Project_path, "data/labelled_masked_images/")
text_files = glob.glob(("data/*.txt"))

## converts xml file in csv format

In [ ]:
def xml_to_csv():
    data = pd.DataFrame(columns=[["filename","xmin","ymin","xmax", "ymax"]])
    for text_file in text_files:
        name_file = text_file.split("/")[-1].split(".")[0][3:]

        with open(text_file,"r") as f:
            text = f.readlines()
        for line in text:
            cordinates = line.split(",")
            xmin, ymin, xmax, ymax = cordinates[0], cordinates[1], cordinates[2], cordinates[3]
            data.loc[-1] = [name_file+".jpg", xmin, ymin, xmax, ymax]  # adding a row
            data.index = data.index + 1  # shifting index
            data = data.sort_index()
    data.to_csv(csv_path)

## convert all input images pixels to white pixels

In [ ]:
def convert_pixel(imag_dir):
    images = glob.glob(os.path.join(image_dir,"*"))
    for image_file in tqdm(images):
        print image_file
        image_name = image_file.split("/")[-1]
        print image_name
        image = cv2.imread(image_file)
        print image.shape
        for x in range(image.shape[0]):
            for y in range(image.shape[1]):

                image[x,y] = (255,255,255)
        cv2.imwrite(os.path.join(images_white_dir, image_name), image)
convert_pixel(image_dir)

## mask the pixels to 0 wherever text is present(given by cordinates for images)

In [ ]:
def mask_text_pixel(csv_path, subdir):
    data = pd.read_csv(csv_path)
    images = glob.glob(os.path.join(images_white_dir, subdir,"*"))

    for image_path in tqdm(images):
    
        image_name = image_path.split("/")[-1]
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        print image_name
        cordinates = data.loc[data['filename'] == image_name,['xmin','xmax','ymin','ymax']]
        print cordinates
        try:
            for index, row in cordinates.iterrows():
                xmin = row['xmin']
                xmax = row['xmax']
                ymin = row['ymin']
                ymax = row['ymax']
                y_len = ymax-ymin
                x_len = xmax-xmin
                print xmin, xmax, "x"
                print ymin,ymax,"y"
                for x in range(xmin,xmax):
                    i = 0
    #                 print "x",x
                    for y in range(ymin,ymax):
                        image[y,x] = (0,0,0)
    #                     print y
                        i=i+1
    #                 xmin = xmin +1

            cv2.imwrite(os.path.join(labelled_images_masked,subdir, image_name), image)
        except:
            print image_name
            print image.shape,"image_shape"
            print 

mask_text_pixel(os.path.join(test_csv_path),"")                         

## rough work

In [ ]:
# def convert_pixel(csv_path, sub_dir):
#     data = pd.read_csv(csv_path)
    
#     for index,row in tqdm(data.iterrows()):
#         image_ext = ""
#         print row['filename']
#         if (not os.path.exists(os.path.join(images_white_dir,sub_dir,row['filename']+image_ext))) and (not os.path.exists(os.path.join(images_white_dir,sub_dir,row['filename']+image_ext))): 
            
#             if os.path.exists(os.path.join(image_dir,sub_dir,row['filename'] + ".jpg")):
# #             if os.path.exists ('/Challenge1_Training_Task12_Images/img_1.jpg'):
#                 image_ext = ".jpg"
#                 print "yes"
#             elif os.path.exists(os.path.join(image_dir, sub_dir, row['filename'] + ".png")):
#                 image_ext = ".png"
#                 print "no"
#             image = cv2.imread(os.path.join(image_dir,row['filename']+image_ext), cv2.IMREAD_COLOR)
            
#             for x in range(image.shape[0]):
#                 for y in range(image.shape[1]):

#                     image[x,y] = (255,255,255)

#             cv2.imwrite(os.path.join(images_white_dir, sub_dir, row['filename'] + image_ext), image)
        
# convert_pixel(test_csv_path,'')     
